# Первичный анализ данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np

In [ ]:
import cv2
from tqdm import tqdm
import torch
import torchvision
random_seed = 42
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

## Загрузка данных

In [ ]:
# загружаем обучающую выборку 
train_data = torchvision.datasets.MNIST(
    "mnist_content", train=True, transform=None, download=True
)

# разделяем обучающую выборку на обучающую и валидационную выборки
# 70% для обучения, 30% для валидации
train_size = int(len(train_data) * 0.7)
valid_size = len(train_data) - train_size
train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
    "mnist_content", train=False, transform=None, download=True
)

In [ ]:
# генерируем имена колонок для пикселей изображения
img_col_names = [f"{i}x{j}" for i in range(1, 29) for j in range(1, 29)]

In [ ]:
train_df = []
stream = tqdm(train_data, desc="Train Data Processing")
for img, label in stream:
    np_img = np.array(img)
    train_df.append([label] + np_img.flatten().tolist())

train_df = pd.DataFrame(train_df, columns=["label"] + img_col_names)
train_df.shape

In [ ]:
valid_df = []
stream = tqdm(valid_data, desc="Valid Data Processing")
for img, label in stream:
    np_img = np.array(img)
    valid_df.append([label] + np_img.flatten().tolist())

valid_df = pd.DataFrame(valid_df, columns=["label"] + img_col_names)
valid_df.shape

In [ ]:
test_df = []
stream = tqdm(test_data, desc="Test Data Processing")
for img, label in stream:
    np_img = np.array(img)
    test_df.append([label] + np_img.flatten().tolist())

test_df = pd.DataFrame(test_df, columns=["label"] + img_col_names)
test_df.shape

## Обучающая выборка

In [ ]:
# Общая информация о данных: размер датасета, тип данных
train_df.info()
# Посмотрим на то как представленны данные 
train_df.head()

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(train_df['label'].unique())

Обучаающая выборка содержит 42,000 строк, 785 колонок. Первая колонка- лэйбл картинки, содержит значения от 0-9. Колонки от 1-785 (1х1 - 28х28) сосдержат значения каждого пиксля изображения. Т.е в датасете представлено 42,000 примеров изображения для обучения 

Рассмотрим информацию о данных для каждой колонки отдельно для того чтобы оценить есть ли пропуски в данных 

In [ ]:
train_df.isnull().sum()

Пропуски в данных отсутсвуют 

Рассмотрим соотношение значений в целевой колонке - колонке лэйблов изображений
Представим информацию в числовом виде и в виде графика 

In [ ]:
# TODO: добавить подписи к осям 
# TODO: добавить название графика 
# TODO: сделать картинку побольше
# TODO: добавить сверху каждого бина число = кол-во лейблов данного класса 
    # это можно сделать через _fig.bar_label(_fig.containers[0])
 
plt.figure(figsize=(8,8))    
plt_1 = plt.bar(sorted(train_df['label'].unique()) ,train_df['label'].value_counts() )
plt.xticks(sorted(train_df['label'].unique()))
plt.title('Hist for Lable column\n Trainig data')
plt.xlabel('Labels')
plt.ylabel('Volume')
plt.bar_label(plt_1,plt_1.datavalues)

plt.show()

Дисбаланса в данных нет, датасет содержит примерно равное количество примеров изображений по каждой цифре

In [ ]:
# Мне кажется, что это не очень понятный график
# может быть просто посмотреть среднее значение, медиану, моду, без графиков, а фисто значения?
@interact 
def choose_pixel(i=widgets.IntSlider(value=1, min=1, max=784)): 
    plt.hist(data=train_df, x=train_df.columns.tolist()[i],bins=25,align='mid')

Рассмотрим средние значения, значения медиан и мод для колонок 1-785

In [ ]:
# Среднее, медиана, мода для каждой колонки
df_train_mean=train_df.iloc[:,1:].mean()
df_train_median=train_df.iloc[:,1:].median()
df_train_mode=train_df.iloc[:,1:].mode()
print(f'Mean: {df_train_mean.head()}\n Median:{df_train_median.head()}\n Mode:{df_train_mode.iloc[:,:5]}')
print(f'Max values\n max mean:{df_train_mean.max()}\n max median {df_train_median.max()}\n max mode {df_train_mode.max().head()}')


Максимальное среднее значение - это 139,3
Значение медиан по колонкам варируется от 0 до 171
В колонках 1-785 преобладает значение 0

Обозначим среднее значение пикселя в изображении для разных цифр как средняя интенсивность.

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_train_labels=pd.DataFrame({'labels':train_df['label'],'intensity':train_df.iloc[:,1:].mean(axis=1)})

# TODO: добавить подписи к осям 
# TODO: добавить название графика 
# TODO: сделать картинку побольше

plt.figure(figsize=(8,8))
plt_2 =plt.bar(df_train_labels['labels'],df_train_labels['intensity'])
plt.xticks(sorted(df_train_labels['labels'].unique()))
plt.title('Average intensity for different digits\n Train data')
plt.xlabel('Digit label')
plt.ylabel('Average intensity')
plt.show()

Из данного графика видно, что самая низкая интенсивность наблюдается у цифры 1 (что связано с особенностями ее написания), самой высокой интенсивностью обладают цифры 0 и 8. 

Графики распределения средней интенсивности по лэйблу

In [ ]:
# TODO: добавить подписи к осям 
# TODO: сделать картинку побольше

for i in range(10):
    plt.figure(figsize=(8,8))
    plt.hist(df_train_labels['intensity'][df_train_labels['labels']==i],bins=85,range=(0,100))
    plt.title(f'Intensity Histohram for {i}\n Train data')
    plt.xlabel('Average intensity')
    plt.ylabel('Count')
    plt.show()

Полученные распределения близки к нормальному распределению, но некоторые графики имеют более высокую дисперсию, чем другие, так как написание цифр у разных людей отличается. Так например у 1 наименьший разброс, в связи с чем можно предположить, что написание 1 имеет меньше всего различий.

Посмотрим на выборочные изображения рукописных цифр 

In [ ]:
@ interact

def show_digits(num=widgets.IntSlider(value=0, min=0, max=9)):
    arr_of_pics=[]
    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=[15, 10])
    for i in range (5):
        j=np.random.choice(train_df[train_df['label']==num].index)
        digit=np.array(train_df.loc[j])
        arr_of_pics.append(np.resize(digit,(28,28)))

    ax[0].imshow(arr_of_pics[0],cmap='gray')
    ax[1].imshow(arr_of_pics[1],cmap='gray')
    ax[2].imshow(arr_of_pics[2],cmap='gray')
    ax[3].imshow(arr_of_pics[3],cmap='gray')
    ax[4].imshow(arr_of_pics[4],cmap='gray')
    

    plt.show()

## Валидационная выборка

In [ ]:
# Общая информация о данных: размер датасета, тип данных
valid_df.info()
# Посмотрим на то как представленны данные 
valid_df.head()

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(valid_df['label'].unique())

In [ ]:
valid_df.isnull().sum()

Данные выборки для валидации имеют такое же представление как и выборка для обучения: первая колонка - это лэйбл картинки, она содержит значения от 0-9, колонки от 1-785 (1х1 - 28х28) сосдержат значения каждого пиксля изображения.
Валидационная выбрка содержит 18,000 изображений.

Пропуски в данных отсутвуют 

In [ ]:
# Оценим количество изображений для каждого лэйбла
plt.figure(figsize=(8,8))    
plt3 = plt.bar(sorted(valid_df['label'].unique()) ,valid_df['label'].value_counts())
plt.xticks(sorted(valid_df['label'].unique()))
plt.title('Hist for Lable column\n Valid data')
plt.xlabel('Labels')
plt.ylabel('Count')
plt.bar_label(plt3,valid_df['label'].value_counts())

plt.show()

Выборка содержит примерно равное количество изображений для каждого лэйбла

Для колонок 1-785 получим среднее значение, медиану и моду

In [ ]:
# Среднее значение, медиана, мода по колонкам 

df_valid_mean=valid_df.iloc[:,1:].mean()
df_valid_median=valid_df.iloc[:,1:].median()
df_valid_mode=valid_df.iloc[:,1:].mode()
print(f'Mean: {df_valid_mean}\n Median:{df_valid_median}\n Mode:{df_valid_mode}')
print(f'Max values\n max mean:{df_valid_mean.max()}\n max median {df_valid_median.max()}\n max mode {df_valid_mode.max()}')

Максимальное среднее значение = 140,1.
Значения медиан по колонкам варируются от 0 до 172.
Моды по колонкам равны 0, следовательно чаще всего в колонках 1-785 встречается значение 0.

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_valid_labels=pd.DataFrame({'labels':valid_df['label'],'intensity':valid_df.iloc[:,1:].mean(axis=1)})

# TODO: добавить подписи к осям 
# TODO: добавить название графика 
# TODO: сделать картинку побольше

plt.figure(figsize=(8,8))
plt_2 =plt.bar(df_valid_labels['labels'],df_valid_labels['intensity'])
plt.xticks(sorted(df_valid_labels['labels'].unique()))
plt.title('Valid data \n Average intensity for different digits')
plt.xlabel('Digit label')
plt.ylabel('Average intensity')

plt.show()

Цифры 0 и 8 имеют наибольшие значения средней интенсивности изображений, а цифры 1 и 4 имеют самые низкие показатели интенсивности

In [ ]:
# TODO: добавить подписи к осям 
# TODO: сделать картинку побольше

for i in range(10):
    plt.figure(figsize=(8,8))
    plt.hist(df_valid_labels['intensity'][df_valid_labels['labels']==i],bins=85,range=(0,100))
    plt.title(f'Intensity Histohram for {i}\n Valid data')
    plt.xlabel('Average intensity')
    plt.ylabel('Count')
    plt.show()

Дисперсия распределения интенсивности зависит от того насколько различается написание цифр

Примеры изображений из валидационной выборки

In [ ]:
@ interact

def show_digits(num=widgets.IntSlider(value=0, min=0, max=9)):
    arr_of_pics=[]
    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=[15, 10])
    for i in range (5):
        j=np.random.choice(valid_df[valid_df['label']==num].index)
        digit=np.array(valid_df.loc[j])
        arr_of_pics.append(np.resize(digit,(28,28)))

    ax[0].imshow(arr_of_pics[0],cmap='gray')
    ax[1].imshow(arr_of_pics[1],cmap='gray')
    ax[2].imshow(arr_of_pics[2],cmap='gray')
    ax[3].imshow(arr_of_pics[3],cmap='gray')
    ax[4].imshow(arr_of_pics[4],cmap='gray')
    

    plt.show()

## Тестовая выборка

In [ ]:
# Общая информация о данных: размер датасета, тип данных
test_df.info()
test_df.head(3)

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(test_df['label'].unique())

In [ ]:
# Кол-во пропусков в данных
test_df.isnull().sum()


Тестовая выбрка содержит 10,000 изображений.
Пропуски в данных отсутсвуют

In [ ]:
# Оценим количество изображений для каждого лэйбла
plt.figure(figsize=(8,8))    
plt4 = plt.bar(sorted(test_df['label'].unique()) ,test_df['label'].value_counts())
plt.xticks(sorted(test_df['label'].unique()))
plt.title('Hist for Lable column\n Test data')
plt.xlabel('Labels')
plt.ylabel('Count')
plt.bar_label(plt4,test_df['label'].value_counts())

plt.show()

In [ ]:
# Среднее значение, медиана, мода по колонкам 

df_test_mean=test_df.iloc[:,1:].mean()
df_test_median=test_df.iloc[:,1:].median()
df_test_mode=test_df.iloc[:,1:].mode()
print(f'Mean: {df_test_mean}\n Median:{df_test_median}\n Mode:{df_test_mode}')
print(f'Max values\n max mean:{df_test_mean.max()}\n max median {df_test_median.max()}\n max mode{df_test_mode.max()}')

Среднее значение варируется от 0 до 142,9.
Значение медианы принимает значение от 0 до 183. 
Чаще всего в колонках встречается значение 0, следовательно большую часть картинки занимает фон.

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_test_labels=pd.DataFrame({'labels':test_df['label'],'intensity':test_df.iloc[:,1:].mean(axis=1)})

# TODO: добавить подписи к осям 
# TODO: добавить название графика 
# TODO: сделать картинку побольше

plt.figure(figsize=(8,8))
plt_2 =plt.bar(df_test_labels['labels'],df_test_labels['intensity'])
plt.xticks(sorted(df_test_labels['labels'].unique()))
plt.title('Test data \n Average intensity for different digits')
plt.xlabel('Digit label')
plt.ylabel('Average intensity')

plt.show()

В тестовой выборке средняя интенсивность изображений цифры 2 стала выше чем в обучающей и валидационной выборке, а средняя интенсивность цифры 7 меньше, чем в обущающей и валидационной выборках; таким образом наибольша интенсивность у цифр 0,2,8;
наименьшая интенсивность у цифр 1,7

In [ ]:
# TODO: добавить подписи к осям 
# TODO: сделать картинку побольше

for i in range(10):
    plt.figure(figsize=(8,8))
    plt.hist(df_test_labels['intensity'][df_test_labels['labels']==i],bins=85,range=(0,100))
    plt.title(f'Intensity Histohram for {i}\n Test data')
    plt.xlabel('Average intensity')
    plt.ylabel('Count')
    plt.show()

Распределения средней интенсивности цифр 2,3,4 имеют несколько вершин, что может говорить о том что для данных цифр имеется несколько различных вариантов их написания

Примеры изображений цифр из тестовой выборки

In [ ]:
@ interact

def show_digits(num=widgets.IntSlider(value=0, min=0, max=9)):
    arr_of_pics=[]
    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=[15, 10])
    for i in range (5):
        j=np.random.choice(test_df[test_df['label']==num].index)
        digit=np.array(test_df.loc[j])
        arr_of_pics.append(np.resize(digit,(28,28)))

    ax[0].imshow(arr_of_pics[0],cmap='gray')
    ax[1].imshow(arr_of_pics[1],cmap='gray')
    ax[2].imshow(arr_of_pics[2],cmap='gray')
    ax[3].imshow(arr_of_pics[3],cmap='gray')
    ax[4].imshow(arr_of_pics[4],cmap='gray')
    

    plt.show()